In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

In [2]:
lih = HamiltonianSmall('LiH', 1.5)
beh2 = HamiltonianSmall('BeH2', 1.3)

h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

water_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')
water_parity = Hamiltonian('H2O_STO3g_14qubits', 'parity')
water_bk = Hamiltonian('H2O_STO3g_14qubits', 'bk')

ammonia_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')
ammonia_parity = Hamiltonian('NH3_STO3g_16qubits', 'parity')
ammonia_bk = Hamiltonian('NH3_STO3g_16qubits', 'bk')

In [3]:
%time lih.pauli_rep.ground()
%time beh2.pauli_rep.ground()
%time h2_jw.pauli_rep.ground()
%time water_jw.pauli_rep.ground()
%time energy, state = ammonia_jw.pauli_rep.ground()

CPU times: user 213 ms, sys: 80.1 ms, total: 293 ms
Wall time: 109 ms
CPU times: user 254 ms, sys: 698 µs, total: 254 ms
Wall time: 254 ms
CPU times: user 541 ms, sys: 31.2 ms, total: 572 ms
Wall time: 414 ms
CPU times: user 23.2 s, sys: 2.46 s, total: 25.6 s
Wall time: 10.2 s
CPU times: user 7min 8s, sys: 2min 1s, total: 9min 10s
Wall time: 4min 25s


In [4]:
from sparse import energy as sparseenergy
from sparse import energy_kron

pr = beh2.pauli_rep
energy, state = pr.ground()
print("energy: ", energy)
%time energy_sparse = sparseenergy(pr, state)
print("energy_sparse: ", energy_sparse)
%time energy_kron = energy_kron(pr, state)
print("energy_kron: ", energy_kron)

energy:  0.8494938395630857
CPU times: user 287 ms, sys: 2.38 ms, total: 290 ms
Wall time: 288 ms
energy_sparse:  0.8494938395630873
CPU times: user 21.7 s, sys: 5.23 ms, total: 21.7 s
Wall time: 21.7 s
energy_kron:  0.8494938395630877


In [5]:
β = pr.local_dists_uniform()
%time pr.variance_local(energy, state, β)

CPU times: user 356 ms, sys: 4.73 ms, total: 361 ms
Wall time: 357 ms


22.470145625464703

In [6]:
print("ell_1: ", pr.variance_ell_1(energy))

β = pr.local_dists_pnorm(1)
print("1_norm: ", pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm(2)
print("2_norm: ", pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm('infinity')
print("max_norm: ", pr.variance_local(energy, state, β))

ell_1:  44.84428759596597
1_norm:  92.75535492849612
2_norm:  84.12338038780376
max_norm:  82.42512285439769


# Variance optimisation (old and wrong, but convex)

If we use a uniform/biased pauli/magic bases algorithm to estimate the energy, and we assume that the state is a product-state (or close to a produce-state) then we end up wanting to minimise:
$$
    \Var[\nu]
    =
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    +
    \sum_{\Qarrow,\Rarrow \in \{I,Z\}^{\otimes n}} \alpha_\Qarrow \alpha_\Rarrow \prod_{i} m_i(1-\delta_{Q_i R_i})
    - \tr(H_0\rho)^2
$$
(Above is the chemistry HF situation, where the product-state is in the $Z$-basis, but this statement wouldn't depend on using $Z$-basis uniformly.)

We end up wanting to minimise:
$$
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    \qquad
    \textrm{subject to}
    \qquad
    \beta_{i,X}+\beta_{i,Y}+\beta_{i,Z}=1 \,\forall i,
    \qquad
    \beta_{i,P}\ge 0
$$

In [7]:
β_optimal = pr.local_dists_optimal()
print(pr.variance_local(energy, state, β_optimal))

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


16.194135745577075


In [8]:
def variances_dict(pauli_rep):
    pr = pauli_rep
    dic = {}
    
    energy, state = pr.ground()
    print("energy :", energy)

    # ell_1
    var = pr.variance_ell_1(energy)
    print("ell 1: ", var)
    dic['ell_1'] = var
    
    # uniform
    β = pr.local_dists_uniform()
    var = pr.variance_local(energy, state, β)
    print("uniform: ", var)
    dic['uniform'] = var
    
    # optimal
    β = pr.local_dists_optimal()
    var = pr.variance_local(energy, state, β)
    print("optimal: ", var)
    dic['optimal'] = var
    
    return dic

In [9]:
variances_ALL = {}
beta_optimal_ALL = {}

In [10]:
name = 'lih'
pr = lih.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -0.6791966848293147
ell 1:  8.826402522940885
uniform:  2.247176209610139
optimal:  1.5743727896616169
CPU times: user 860 ms, sys: 77.8 ms, total: 937 ms
Wall time: 361 ms


In [11]:
name = 'beh2'
pr = beh2.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : 0.8494938395630848
ell 1:  44.84428759596597
uniform:  22.470145625464596
optimal:  16.19413574557702
CPU times: user 4.67 s, sys: 394 ms, total: 5.07 s
Wall time: 1.82 s


In [12]:
name = 'h2_jw'
pr = h2_jw.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207596
ell 1:  119.6790600190591
uniform:  51.3998202138761
optimal:  17.7420719368114
CPU times: user 11 s, sys: 1.03 s, total: 12 s
Wall time: 3.37 s


In [13]:
name = 'h2_parity'
pr = h2_parity.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207574
ell 1:  119.6790600190592
uniform:  70.76147441962269
optimal:  18.924750112035017
CPU times: user 35.8 s, sys: 3.56 s, total: 39.3 s
Wall time: 8.81 s


In [14]:
name = 'h2_bk'
pr = h2_bk.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.86086055552077
ell 1:  119.67906001905936
uniform:  168.86600324405498
optimal:  19.516932420931546
CPU times: user 20.9 s, sys: 2.15 s, total: 23.1 s
Wall time: 5.45 s


In [15]:
pr = water_jw.pauli_rep
%time β = pr.local_dists_optimal()
β

CPU times: user 6min 53s, sys: 1min 1s, total: 7min 54s
Wall time: 59.2 s


{0: [0.43257858556565915, 0.43257858722135467, 0.1348428272129862],
 1: [0.24098007537433425, 0.2409800708577077, 0.518039853767958],
 2: [0.13870541903721406, 0.13870542588015417, 0.7225891550826317],
 3: [0.13957872152763764, 0.13957871134125624, 0.7208425671311062],
 4: [0.16984472160573807, 0.1698447243114196, 0.6603105540828423],
 5: [0.2675208449775755, 0.2675208434379076, 0.464958311584517],
 6: [0.11647067744295292, 0.11647067819430176, 0.7670586443627454],
 7: [0.43257857708372555, 0.43257860312684154, 0.13484281978943294],
 8: [0.24098005034367995, 0.24098008250406633, 0.5180398671522537],
 9: [0.13870542248842968, 0.13870542724539398, 0.7225891502661764],
 10: [0.13957872657623138, 0.13957872195005588, 0.7208425514737127],
 11: [0.16984475333796817, 0.16984473040786618, 0.6603105162541657],
 12: [0.2675208758292949, 0.26752084565650264, 0.46495827851420257],
 13: [0.11647066971784335, 0.1164706702788042, 0.7670586600033525]}

In [16]:
%time energy, state = pr.ground()

CPU times: user 22.8 s, sys: 2.42 s, total: 25.2 s
Wall time: 9.86 s


In [17]:
%time var = pr.variance_ell_1(energy)
print("ell 1: ", var)

CPU times: user 70 µs, sys: 10 µs, total: 80 µs
Wall time: 12.9 µs
ell 1:  4363.497773126058


In [18]:
%time var = pr.variance_local(energy, state, pr.local_dists_uniform())
print("uniform: ", var)

CPU times: user 2min 57s, sys: 1.89 s, total: 2min 59s
Wall time: 2min 58s
uniform:  2839.0394682188453


In [19]:
%time var = pr.variance_local(energy, state, β)
print("optimal: ", var)

CPU times: user 2min 58s, sys: 1.72 s, total: 3min
Wall time: 3min
optimal:  257.5450993726922
